In [1]:
import os

In [2]:
%pwd

'/Users/karthikaselvaraj/Desktop/test/Sports_Classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/karthikaselvaraj/Desktop/test/Sports_Classification'

In [5]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [6]:
from sports_classifier.constants import *
from sports_classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
# configuration manager
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([Path(model_ckpt_dir),
                           Path(config.tensorboard_root_log_dir)])
        
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> PrepareTrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "trainData")
        
        create_directories([
            Path(training.root_dir)
        ])

        training_config = PrepareTrainingConfig(
                root_dir = Path(training.root_dir),
                trained_model_path = Path(training.trained_model_path),
                updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
                training_data = Path(training_data),
                params_epochs = params.EPOCHS,
                params_batch_size = params.BATCH_SIZE,
                params_is_augmentation = params.AUGUMENTATION,
                params_image_size = params.IMAGE_SIZE
        )

        return training_config




In [8]:
# components
# callback component 
import time

class PrepareCallback:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= "artifacts/prepare_callbacks/checkpoint_dir/model.h5",
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [9]:
# training component

import os
import time
import tensorflow as tf 
import urllib.request as request
from zipfile import ZipFile

In [10]:
class Training:
    def __init__(self, config: PrepareTrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self): # splitting data into train & validation
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

In [11]:
# pipeline
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list = callback_list
    )
except Exception as e:
    raise e

[2023-11-23 15:07:17,637: INFO: common: yaml file config/config.yaml loaded successfully]
[2023-11-23 15:07:17,640: INFO: common: yaml file params.yaml loaded successfully]
[2023-11-23 15:07:17,641: INFO: common: Created directory at : artifacts]
[2023-11-23 15:07:17,641: INFO: common: Created directory at : artifacts/prepare_callbacks/checkpoint_dir]
[2023-11-23 15:07:17,642: INFO: common: Created directory at : artifacts/prepare_callbacks/tensorboard_log_dir]
[2023-11-23 15:07:17,645: INFO: common: Created directory at : artifacts/training]
[2023-11-23 15:07:17,833: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.SGD`.]
Found 192 images belonging to 8 classes.
Found 768 images belonging to 8 classes.
Epoch 1/10
48/48 [==============================] - 102s 2s/step - loss: 24.9374 - accuracy: 0.1458 - val_loss: 30.7751 - val_accuracy: 0.1250

/Users/karthikaselvaraj/anaconda3/envs/sports1/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


48/48 [==============================] - 114s 2s/step - loss: 14.7449 - accuracy: 0.2643 - val_loss: 14.8131 - val_accuracy: 0.4219
Epoch 3/10
48/48 [==============================] - 115s 2s/step - loss: 11.7217 - accuracy: 0.3307 - val_loss: 6.6118 - val_accuracy: 0.4062
Epoch 4/10
48/48 [==============================] - 124s 3s/step - loss: 7.2026 - accuracy: 0.4440 - val_loss: 6.3065 - val_accuracy: 0.4531
Epoch 5/10
48/48 [==============================] - 125s 3s/step - loss: 7.1977 - accuracy: 0.4297 - val_loss: 4.5175 - val_accuracy: 0.5365
Epoch 6/10
48/48 [==============================] - 131s 3s/step - loss: 5.5356 - accuracy: 0.4805 - val_loss: 7.7063 - val_accuracy: 0.5000
Epoch 7/10
48/48 [==============================] - 135s 3s/step - loss: 4.4168 - accuracy: 0.5495 - val_loss: 1.7008 - val_accuracy: 0.6875
Epoch 8/10
48/48 [==============================] - 133s 3s/step - loss: 3.5869 - accuracy: 0.5807 - val_loss: 9.3356 - val_accuracy: 0.5417
Epoch 9/10
48/48 [===